In [7]:
import pyodbc
import pandas as pd  # Fixed typo: was 'pf', should be 'pd'
from datetime import datetime, date

# Database connection
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-RJ6N7AA\MSSQLSERVER1;"
    "DATABASE=scd;"
    "UID=sa;"
    "PWD=1234567890;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

SCD Type  0 - No Changes Allowed

In [8]:
# Create SCD Type 0 table
cursor.execute("""
CREATE TABLE scd0_customer (
    customer_id INT PRIMARY KEY,
    name NVARCHAR(100),
    birth_date DATE,
    created_date DATETIME DEFAULT GETDATE()
)
""")

# Insert initial data
cursor.execute("""
INSERT INTO scd0_customer (customer_id, name, birth_date) 
VALUES (1, 'John Smith', '1990-05-15')
""")
conn.commit()

# Try to update (this would be rejected in real SCD Type 0)
print("SCD Type 0: Birth date never changes!")

SCD Type 0: Birth date never changes!


SCD Type 1 - Overwrite

In [9]:
# Create SCD Type 1 table
cursor.execute("""
CREATE TABLE scd1_customer (
    customer_id INT PRIMARY KEY,
    name NVARCHAR(100),
    phone NVARCHAR(20),
    last_updated DATETIME DEFAULT GETDATE()
)
""")

# Insert initial data
cursor.execute("""
INSERT INTO scd1_customer (customer_id, name, phone) 
VALUES (1, 'John Smith', '123-456-7890')
""")
conn.commit()

# Update phone number (overwrites old value)
cursor.execute("""
UPDATE scd1_customer 
SET phone = '987-654-3210', last_updated = GETDATE() 
WHERE customer_id = 1
""")
conn.commit()

# Check result
df = pd.read_sql("SELECT * FROM scd1_customer WHERE customer_id = 1", conn)
print("SCD Type 1 Result:")
print(df)
print("Old phone number is lost forever!")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'scd1_customer' in the database. (2714) (SQLExecDirectW)")